# Stacked autoencoder

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

In [3]:
mnist = input_data.read_data_sets("MNIST_data/")
trX, trY, teX, teY = mnist.train.images, mnist.train.labels,mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
class StackedAutoEncoder(object):
    def __init__(self, list1, eta = 0.02):
        """
        list1: [input_dimension, hidden_layer_1, ....,hidden_layer_n]
        """
        N = len(list1)-1
        self._m = list1[0]
        self.learning_rate = eta
        # Create the Computational graph
        self._W = {}
        self._b = {}
        self._X = {}
        self._X['0'] = tf.placeholder('float', [None, list1[0]])
        for i in range(N):
            layer = '{0}'.format(i+1)
            print('AutoEncoder Layer {0}: {1} --> {2}'.format(layer, list1[i], list1[i+1]))
            self._W['E' + layer] = tf.Variable(tf.random_normal(shape=(list1[i],list1[i+1])),name='WtsEncoder'+layer)
            self._b['E'+ layer] =tf.Variable(np.zeros(list1[i+1]).astype(np.float32),name='BiasEncoder'+layer)
            self._X[layer] = tf.placeholder('float', [None, list1[i+1]])
            self._W['D' + layer] = tf.transpose(self._W['E' + layer]) # Shared weights
            self._b['D' + layer] =tf.Variable(np.zeros(list1[i]).astype(np.float32),name='BiasDecoder' +layer)
        # Placeholder for inputs
        self._X_noisy = tf.placeholder('float', [None, self._m])
        self.train_ops = {}
        self.out = {}
        for i in range(N):
            layer = '{0}'.format(i+1)
            prev_layer = '{0}'.format(i)
            opt = self.pretrain(self._X[prev_layer], layer)
            self.train_ops[layer] = opt
            self.out[layer] = self.one_pass(self._X[prev_layer], self._W['E'+layer],
            self._b['E'+layer], self._b['D'+layer])